# Contents

In this notebook, we will learn how to load the PKIS2 dataset into memory using a `DatasetProvider` object. Then we will apply some standard featurization to obtain ML-compatible representations, and use a simple model to compute some activity predictions.

1. [X] Loading the data
2. [X] Featurizing the data
3. [X] Exporting the featurized data to PyTorch
4. [X] Build and train the model
5. [ ] Analyze results nicely

Disable stereochemistry _warnings_ generated by `openforcefield`.

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [2]:
import warnings
warnings.simplefilter("ignore") 
import logging
logging.basicConfig(level=logging.ERROR)
import numpy as np

# 1. Loading the data as a DatasetProvider

In [3]:
from kinoml.datasets.kinomescan.pkis2 import PKIS2DatasetProvider

RDKit WARNING: [18:28:03] Enabling RDKit 2019.09.2 jupyter extensions


Let's initialize the PKIS2 dataset provider. Instead of a regular `ClassName()` instantiation, we need to use `.from_source()` (with convenient default arguments).

__Why?__

This due to the design of `BaseDatasetProvider.__init__`, which expects a list of `BaseMeasurement` objects (or subclasses of). 

* A `BaseMeasurement` class is normally subclassed by more specific measurements (like `BaseMeasurement`), but the design is the same. It takes:
  * `values`: an array of numeric values (single measurements are _arrayfied_ into single-element arrays). This can be replicates of the value for statistical purposes, or under different concentrations of a reactant?
  * `conditions`: instance of `AssayConditions`. This class provides all the properties required to reproduce the experiment (say `pH`, `temperature`, `concentration`, etc). This should be paired (somehow) to the dimensionality of `values`, but I haven't though much of that yet.
  * `system`: instance of a `System` class or subclass. The subclasses can restrict which type of `MolecularComponent` objects are allowed (e.g. `ProteinLigandComplex` only takes a `Protein` and a `Ligand`).
* A `System` is abstract enough to not impose any restrictions on the composition, but its subclasses can be. 
  * This is the case of the `Complex` object, which requires at least one `BaseProtein` and one `BaseLigand` objects.
* The `MolecularComponent` class is the base object all proteins and ligands, regardless their representation (e.g. sequence vs 3D structure, smiles vs molecular graph). `MolecularComponent` is immediately subclassed by:
  * `BaseProtein`, the abstract model which is subclassed by more concrete classes, like `AminoAcidSequence` and `ProteinStructure`.
  * `BaseLigand`, the abstract model which is subclassed by more concrete classes, like `Ligand` (based on `openforcefield.topology.Molecule`).


Anyway, all those details are not needed to start using the provider. Right now there's no lazy behavior, so it will take _a bit_ to build all sequences and ligands. In my machine, it's about 12 seconds for all 160K datapoints.

In [4]:
%%time
pkis2 = PKIS2DatasetProvider.from_source()

CPU times: user 20 s, sys: 392 ms, total: 20.4 s
Wall time: 20.4 s


You can export a convenient dataframe with this method. Take into account this is just using the default implementation in the base class, which relies on the different `__repr__` methods and `.name` attributes of the objects involved. For prettier dataframes, one can always subclass `to_dataframe` to provide a better presentation.

In [6]:
df = pkis2.to_dataframe()
df

,Systems,n_components,PercentageDisplacementMeasurement
0,AAK1 & Clc1cccc(Cn2c(nn3c2nc(cc3=O)N2CCOCC2)C2...,2,14.0
1,ABL1-nonphosphorylated & Clc1cccc(Cn2c(nn3c2nc...,2,28.0
2,ABL1-nonphosphorylated & Clc1cccc(Cn2c(nn3c2nc...,2,20.0
3,ABL2 & Clc1cccc(Cn2c(nn3c2nc(cc3=O)N2CCOCC2)C2...,2,5.0
4,ACVR1 & Clc1cccc(Cn2c(nn3c2nc(cc3=O)N2CCOCC2)C...,2,0.0
...,...,...,...
261865,ZAP70 & CCn1c(nc2c(nc(OC[C@H](N)c3ccccc3)cc12)...,2,0.0
261866,p38-alpha & CCn1c(nc2c(nc(OC[C@H](N)c3ccccc3)c...,2,0.0
261867,p38-beta & CCn1c(nc2c(nc(OC[C@H](N)c3ccccc3)cc...,2,0.0
261868,p38-delta & CCn1c(nc2c(nc(OC[C@H](N)c3ccccc3)c...,2,0.0


Although we have 260K measurements, there are roughly 250K _systems_, comprised of a reduced number of ligands and proteins

In [12]:
print("Measurements:", len(pkis2.measurements))
print("Systems:", len(pkis2.systems))
print("Ligands:",len(set([s.ligand for s in pkis2.systems])))
print("Proteins:", len(set([s.protein for s in pkis2.systems])))

Measurements: 261870
Systems: 257920
Ligands: 640
Proteins: 403


Notice how the string representations try to be a bit informative.

In [7]:
pkis2

<PKIS2DatasetProvider with 261870 PercentageDisplacementMeasurement measurements and 257920 systems>

In [8]:
one_random_system = next(iter(pkis2.systems))
one_random_system

<ProteinLigandComplex with 2 components (<AminoAcidSequence name=NEK5>, <Ligand name=COC(=O)c1cccc(NC(=O)NC2CCN(C2)c2ccnc(Nc3ccc(F)cc3)n2)c1>)>

Some areas do need improvement... this is how you get all the entries that have wild-type kinases (all of them in PKIS2). Maybe some Django-style queries?

In [9]:
wt = [ms for ms in pkis2.measurements if ms.system.protein.metadata["mutations"] is None]
wt_provider = PKIS2DatasetProvider(measurements=wt)
wt_provider

<PKIS2DatasetProvider with 261870 PercentageDisplacementMeasurement measurements and 257920 systems>

#  2. Featurizing the data

We will be using:

- MorganFingerprint n=2048 bits, r=2
- OneHotEncoding of protein sequence
- ... or composition of binding site

An isolated featurizer takes one system and returns the raw data:

In [10]:
from kinoml.features.ligand import MorganFingerprintFeaturizer
morgan_featurizer = MorganFingerprintFeaturizer(nbits=1024, radius=2)
syst = morgan_featurizer.featurize(next(iter(pkis2.systems)))
print(syst.featurizations[morgan_featurizer.name].shape, *syst.featurizations[morgan_featurizer.name][:75], "...")

(1024,) 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...


In the context of a dataset provider, each system will store that raw data in an internal dictionary (`.featurizations`) for _each_ system. Without caching, this would take ~30 minutes, given the huge amount of duplication in the dataset. Thanks to LRU caching at the `featurizer` level, each `Ligand` is only featurized once!

In [11]:
from kinoml.features.protein import OneHotEncodedSequenceFeaturizer
sequence_featurizer = OneHotEncodedSequenceFeaturizer()
syst = sequence_featurizer.featurize(next(iter(pkis2.systems)))
print(syst.featurizations[sequence_featurizer.name].shape, *syst.featurizations[sequence_featurizer.name][0], "...")

(20, 279) 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.

Since sequence length can differ along the kinase set, we need to pad to the maximum length.

In [12]:
max_length = len(pkis2.systems[0].protein.sequence)
best = pkis2.systems[0]
for system in pkis2.systems[1:]:
    this_length = len(system.protein.sequence)
    if this_length > max_length:
        max_length = this_length
        best = system
print(best, "has", max_length, "aas")

<ProteinLigandComplex with 2 components (<AminoAcidSequence name=PIK3C2B>, <Ligand name=COC(=O)c1cccc(NC(=O)NC2CCN(C2)c2ccnc(Nc3ccc(F)cc3)n2)c1>)> has 1634 aas


We could use annotations at UniProt to clip the useful domains. It might be too aggressive, but it's a good start.

We can also try to align to Dunbrack's MSA: https://www.nature.com/articles/s41598-019-56499-4

In [13]:
from kinoml.features.core import PadFeaturizer, Pipeline
padded_featurizer = PadFeaturizer((max_length,), key=sequence_featurizer.__class__.__name__)
padded_sequence_featurizer = Pipeline([sequence_featurizer, padded_featurizer])
syst = padded_sequence_featurizer.featurize(next(iter(pkis2.systems)))
print(syst.featurizations[padded_sequence_featurizer.name].shape, *syst.featurizations[padded_sequence_featurizer.name][0][:100], "...")

(1634, 1893) 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ...


In [14]:
syst.featurizations

{'MorganFingerprintFeaturizer': array([0, 0, 0, ..., 0, 0, 0], dtype=uint8),
 'OneHotEncodedSequenceFeaturizer': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 'Pipeline([OneHotEncodedSequenceFeaturizer, PadFeaturizer])': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])}

For this exercise, we will use a simpler protein featurization, like hashed name, for easy concatenation.

In [15]:
from kinoml.features.core import HashFeaturizer

hashed_sequence_featurizer = HashFeaturizer(("protein", "sequence"))
syst = hashed_sequence_featurizer.featurize(next(iter(pkis2.systems)))
print(syst.featurizations[hashed_sequence_featurizer.name])

[0.56476528]


In [16]:
from kinoml.features.core import Concatenated
concat_featurizers = Concatenated([morgan_featurizer, hashed_sequence_featurizer], axis=0)

In [17]:
%%time
pkis2.featurize(concat_featurizers)

Featurizing systems...: 100%|██████████| 257920/257920 [00:28<00:00, 8987.08it/s] 

CPU times: user 27.3 s, sys: 2.6 s, total: 29.9 s
Wall time: 28.8 s


In [18]:
for system in pkis2.systems[:5]:
    print(system, "...\n  ", system.featurizations, "\n")

<ProteinLigandComplex with 2 components (<AminoAcidSequence name=NEK5>, <Ligand name=COC(=O)c1cccc(NC(=O)NC2CCN(C2)c2ccnc(Nc3ccc(F)cc3)n2)c1>)> ...
   {'MorganFingerprintFeaturizer': array([0, 0, 0, ..., 0, 0, 0], dtype=uint8), 'OneHotEncodedSequenceFeaturizer': array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), 'Pipeline([OneHotEncodedSequenceFeaturizer, PadFeaturizer])': array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), 'HashFeaturizer': array([0.56476528]), 'Concatenated([MorganFingerprintFeaturizer, HashFeaturizer])': array([0.        , 0.        , 0.        , ..., 0.        , 0.        ,
       0.56476528

# 3. Export to PyTorch

In [19]:
dataset = pkis2.to_pytorch()
dataset

This pytorch dataset implements the `Dataset` protocol and provides two attributes: `measurements` and (featurized) `systems`:

In [20]:
dataset.measurements

array([14., 28., 20., ...,  0.,  0., 34.])

In [21]:
print(dataset.systems[0].shape)

(1025,)


**TODO**: Look into specifying datatypes per featurizer to use memory more efficiently.

## 3.1 Ensuring we have an adequate observation model
The underlying measurement type common to _all_ measurements contains an `observation_model` method that returns a dispatched callable, configurable per backend (default=pytorch).

In [22]:
pct_displacement_model = pkis2.measurement_type.observation_model(backend="pytorch")
pct_displacement_model??

Signature: pct_displacement_model(values, inhibitor_conc=1e-06, **kwargs)
Docstring: <no docstring>
Source:   
    @staticmethod
    def _observation_model_pytorch(values, inhibitor_conc=1e-6, **kwargs):
        import torch

        # values = torch.from_numpy(values)
        return 100 / (1 + torch.exp(values) / inhibitor_conc)
File:      ~/devel/py/openkinome/kinoml/kinoml/core/measurements.py
Type:      function


The `observation_model` function expects native objects to their dataset (or numpy arrays):

# 4. Building and training the model

- `DNNModel`

In [23]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class NeuralNetworkRegression(nn.Module):
    """
    Builds a Pytorch model (a Dense Neural Network) and a feed-forward pass
    """
    def __init__(self, input_size=1024, hidden_size=100, output_size=1):
        super(NeuralNetworkRegression, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        self.fully_connected_1 = nn.Linear(self.input_size, self.hidden_size) # Fully connected layer 
        self.fully_connected_out = nn.Linear(self.hidden_size, self.output_size) # Output

    def forward(self, x):
        """
        Defines the foward pass for a given input 'x'
        """
        x = F.relu(self.fully_connected_1(x)) # Activations are ReLU
        return self.fully_connected_out(x)


## 4.1 Optimization loop

In [25]:
# Use DataLoader for minibatches
loader = dataset.as_dataloader(batch_size=512)

model = NeuralNetworkRegression(input_size=dataset.systems[0].shape[0])
optimizer = torch.optim.DuD(model.parameters(), lr=0.001)
loss_function = nn.MSELoss() # Mean squared error

nb_epoch = 100
loss_timeseries = []
for epoch in range(nb_epoch):
    
    cumulative_loss = 0
    for i, (x, y) in enumerate(loader, 1):
        # Clear gradients
        
        optimizer.zero_grad()
        
        # Obtain model prediction given model input
        # x.requires_grad_()
        delta_g = model(x)

        # with observation model
        # with torch.no_grad():
        prediction = pct_displacement_model(delta_g)
        loss = loss_function(prediction, y)

        # Obtain loss for the predicted output
        cumulative_loss += loss.item()

        # Gradients w.r.t to parameters
        loss.backward()

        # Optimizer
        optimizer.step()
    loss_timeseries.append(cumulative_loss)
    if epoch % 10 == 0:
        print(f"epoch {epoch} : loss {loss_timeseries[-1]}")
print("Done!")

epoch 0 : loss 344249.93074798584
epoch 10 : loss 286994.50827789307
epoch 20 : loss 264993.7321166992
epoch 30 : loss 258397.48007202148
epoch 40 : loss 260831.3882522583
epoch 50 : loss 254199.86698150635
epoch 60 : loss 252250.46099853516
epoch 70 : loss 252102.4603881836
epoch 80 : loss 252873.8337020874
epoch 90 : loss 251525.9186630249
Done!


# 5. Analyze results

In [ ]:
from matplotlib import pyplot as plt
plt.plot(loss_timeseries)